This tutorial assumes that you have installed NeatMS and familiarised yourself with the tool through the documentation at https://readthedocs.org/NeatMS. 
Example data and the default model used here are available on [NeatMS github repository](https://github.com/bihealth/NeatMS). The example data is composed of 3 sample files only.

---

# 1. Setting log output (Jupyter notebook specific)

NeatMS uses python standard logging API to facilitate its integration and maintenance in data processing workflow (*e.g*. [galaxy](https://galaxyproject.org/), [snakemake](https://snakemake.readthedocs.io/en/stable/)). The following code's only purpose is to redirect the logs to the standard output for this tutorial. 

For more information about python logging API, please see the [official documentation](https://docs.python.org/3.6/library/logging.html). 

In [ ]:
import sys
import logging
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

---

# 2. Import NeatMS

Importing NeatMS is as simple as importing any python package.

In [ ]:
import NeatMS as ntms

---

# 3. Creat experiment object and load data

Let's create a NeatMS experiment object which will automatically load the raw data and the aligned/unaligned features. Set the `raw_data_folder_path` and the `feature_table_path` arguments, both absolute and relative path (from this notebook) are accepted.

In [ ]:
raw_data_folder_path = '../../data/test_data/mzML/'
# Using peaks that have been aligned across samples
feature_table_path = '../../data/test_data/aligned_features.csv'
# Using unaligned peaks (One individual peak table for each sample)
# feature_table_path = '../data/test_data/unaligned_features/'
# This is important for NeatMS to read the feature table correctly
input_data = 'mzmine'

experiment = ntms.Experiment(raw_data_folder_path, feature_table_path, input_data)

---

# 4. Data exploration

Here are some simple examples on how you can explore MS data using NeatMS.

We can already explore the data using NeatMS module. For example, let's print sample names and the number of features present in each of them.

In [ ]:
for sample in experiment.samples:
    print('Sample {} : {} peaks'.format(sample.name,len(sample.feature_list)))

---

Let's go further and print the number of feature present in 1, 2, and all 3 samples.

In [ ]:
from  collections import Counter
exp = experiment
sizes = []
print("# Feature collection:",len(exp.feature_tables[0].feature_collection_list))

for consensus_feature in exp.feature_tables[0].feature_collection_list:
    sizes.append(len(consensus_feature.feature_list))

c = Counter(sizes)
print("Number of consensus features:")
for size, count in c.most_common():
    print("   of size %2d : %6d" % (size, count))
print("        total : %6d" % len(exp.feature_tables[0].feature_collection_list)) 

---

# 5. Neural network handler object

Time to create a neural network handler object which will allow us to load an existing model and run it on our data. We just need to pass the `experiment` object to the `NN_handler` so it has access to the data.

In [ ]:
nn_handler = ntms.NN_handler(experiment)

---

Let's now load the model, the default model is available on NeatMS github repository.

In [ ]:
# Adjust the model path (relative and absolute path are both accepted)
# Note that model #2 has been tuned to a specific dataset using transfer learning
# Here we use the first default base model
model_path = "../../data/model/neatms_default_model.h5"
nn_handler.create_model(model = model_path)

---

The `threshold` parameter needs to be set to predict the peaks present in the dataset. The threshold value for the default NeatMS model distributed on the github repository is `0.22`.

When using another model, the threshold value should be given by the person who trained the model. 

> Threshold for model number 2 is `0.20`.

Note: Changing this threshold may have a major impact on the prediction, please read carefully the documentation before adjusting this parameter.

In [ ]:
# Set the threshold to 0.22
threshold=0.22
# Run the prediction
nn_handler.predict_peaks(threshold)

---

# 6. Exploring results

Before exporting the data, looking through the results can help to decide on the tuning of the parameters of the export function. 

The code below is similar to the one that prints out the number of peaks present in a certain number of samples (section 4), except that we now have one more level of complexity, the predicted label. 

In [ ]:
from  collections import Counter
exp = experiment
hq_sizes = []
lq_sizes = []
n_sizes = []
sizes = []
print("# Feature collection:",len(exp.feature_tables[0].feature_collection_list))
for consensus_feature in exp.feature_tables[0].feature_collection_list:
    hq_size = 0
    lq_size = 0
    n_size = 0
    for feature in consensus_feature.feature_list:
        for peak in feature.peak_list:
            if peak.valid:
                if peak.prediction.label == "High_quality":
                    hq_size += 1
                if peak.prediction.label == "Low_quality":
                    lq_size += 1
                if peak.prediction.label == "Noise":
                    n_size += 1

    hq_sizes.append(hq_size)
    lq_sizes.append(lq_size)
    n_sizes.append(n_size)
    sizes.append(len(consensus_feature.feature_list))

c = Counter(hq_sizes)
print("\nNumber of consensus features labeled as 'High quality':")
for size, count in c.most_common():
    print("   of size %2d : %6d" % (size, count))
print("        total : %6d" % len(exp.feature_tables[0].feature_collection_list))

c = Counter(lq_sizes)
print("\nNumber of consensus features labeled as 'Low quality':")
for size, count in c.most_common():
    print("   of size %2d : %6d" % (size, count))
print("        total : %6d" % len(exp.feature_tables[0].feature_collection_list))

c = Counter(n_sizes)
print("\nNumber of consensus features labeled as 'Noise':")
for size, count in c.most_common():
    print("   of size %2d : %6d" % (size, count))
print("        total : %6d" % len(exp.feature_tables[0].feature_collection_list))

We now know that 1130 features are present in all 3 samples in `High_quality`. We could decide to only export those features and be very restrictive, or we can be a bit more conservative and allow `Low_quality` features to be exported under certain conditions. We will see how to do that next using the export function. 

---

# 7. Export results

By default, the export function exports all peaks without filtering, this is our prefered option as it allows keep all the information and filter peaks based on their quality at a later stage using your own criteria (and script).

In [ ]:
filename = 'neatms_export.csv'

experiment.export_csv(filename)

You can also export the data as a pandas dataframe, this is particularly useful when using NeatMS in a python based data analysis workflow.

In [ ]:
# We create the dataframe using this function
NeatMS_output_df = experiment.export_to_dataframe()
# And display it
NeatMS_output_df

## Export properties

You can change the information that you export easily using the `export_properties` argument. Here is an example, and the full list of properties that can be exported

> Note that the list of properties that can be exported depends on the input data. For example, the `sn` property is only available when the data comes from XCMS (and corresponds to XCMS `sn` value), the same applies to `area_bc` which corresponds to XCMS `intb` value. When a property is not available, it will be set to None in the output dataframe. However, when you ask for a property that does not exists, it will simply be ignored. 

As an example, let's add the peak start and end retention time to the dataframe

In [ ]:
# We add those specific properties to the export list
# Default properties will be overwritten, so make sure to add them to the list as well
export_properties = ["rt", "mz", "height", "area", "label", "peak_rt_start", "peak_rt_end"]

# Here is the full list of available properties that you can export
# ["rt", "mz", "height", "area", "label", "peak_rt_start", "peak_rt_end", "peak_mz_min", "peak_mz_max", "area_bc", "sn"]

NeatMS_output_df = experiment.export_to_dataframe(export_properties = export_properties)

NeatMS_output_df

Finally, the function to export the data to a `.csv` file works the same way. 

In [ ]:
filename = 'neatms_export_with_extra_properties.csv'

experiment.export_csv(filename, export_properties = export_properties)

## Filtering options

The export functions are very flexible and allows precise filtering of the peaks to be exported. Please refer to the documentation for details.

For example, the code below exports in a `.csv` file the `height` of the peaks labelled with `High_quality` and `Low_quality`, under the condition that it is present in `High_quality` in a minimum of 2 out of 3 samples. Peaks that do not meet this requirement are discarded. `Noise` labelled peaks are not exported.  


In [ ]:
filename = 'neatms_export.csv'

min_group_classes = ["High_quality"]
min_group_size = 0.66

export_classes = ["High_quality", "Low_quality"]

export_properties = ["rt", "mz", "height", "label"]

experiment.export_csv(filename, export_classes = export_classes, min_group_classes = min_group_classes, min_group_size = min_group_size, export_properties = export_properties)


---

That is all for this tutorial, by now you should be able to load your data into NeatMS and classify the peaks using a pretrained model. For more details on specific function, you can check out the documentation at https://readthedocs.org/NeatMS.

If you want to train your own model to better fit your data, take a look at the advanced section of the documentation and check out the advanced tutorial.

Thank you for using NeatMS!

# 8. Extra: Example code to plot peaks

As an example, let's extract all `High_quality` peaks from one sample.

> Note: You need to have `matplotlib` installed to run this code

In [ ]:
labels = ["High_quality","Low_quality","Noise"]
# Change the index to plot peaks with different label
label = labels[0]

sample = experiment.samples[0]

peak_count = 0
peak_list = []
for peak in sample.peak_list:
    # This 'valid' statement is important, hotfix for a known issue    
    if peak.valid:
        if peak.prediction.label == label:
            peak_list.append(peak)
print('Number of {} peaks in {}: {}'.format(label,sample.name,len(peak_list)))

Now let's plot 2 peaks (plotting too many take a while, consider saving peaks image files rather than displaying them in a notebook if you want to look at all peaks).

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

for i in range(70,72):
    peak = peak_list[i]
    chromatogram = peak.get_chromatogram(1)
    print(i)
    plt.plot(chromatogram[0], chromatogram[1])
    plt.show()